In [1]:
from ucimlrepo import fetch_ucirepo 
import joblib
  
# fetch dataset 
adult = fetch_ucirepo(id=2) 
  
# data (as pandas dataframes) 
X = adult.data.features 
y = adult.data.targets 

In [3]:
## cargar modelo

modelo=joblib.load('pipeline_total.gz')
modelo

Pipeline(steps=[('preprocesador',
                 ColumnTransformer(transformers=[('num',
                                                  ColumnTransformer(transformers=[('escalar',
                                                                                   StandardScaler(),
                                                                                   Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object', name='name'))]),
                                                  Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week'],
      dtype='object',...
                                                                                   Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object', name='name'))]),
                                                  Index(['workclass', 'education', 'marital-status', 'occupation',
       'relationship', 'race', 'sex', 'native-country'],
      dtype='object', name='name'))])),
                ('modelo',
                 RandomForestClassifier(max_depth=20, min_samples_split=10,
                                        random_state=42))])

In [9]:
datos_testeo=X.sample(1)
datos_testeo

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country
3419,39,Private,346478,Some-college,10,Never-married,Exec-managerial,Not-in-family,White,Male,0,0,45,United-States


In [25]:
datos_json = {"age":{"3419":39},
         "workclass":{"3419":"Private"},
         "fnlwgt":{"3419":346478},
         "education":{"3419":"Some-college"},
         "education-num":{"3419":10},
         "marital-status":{"3419":"Never-married"},
         "occupation":{"3419":"Exec-managerial"},
         "relationship":{"3419":"Not-in-family"},
         "race":{"3419":"White"},"sex":{"3419":"Male"},
         "capital-gain":{"3419":0},"capital-loss":{"3419":0},
         "hours-per-week":{"3419":45},
         "native-country":{"3419":"United-States"}}
datos_transformados = {k: v["3419"] for k, v in datos_json.items()}
datos_transformados

{'age': 39,
 'workclass': 'Private',
 'fnlwgt': 346478,
 'education': 'Some-college',
 'education-num': 10,
 'marital-status': 'Never-married',
 'occupation': 'Exec-managerial',
 'relationship': 'Not-in-family',
 'race': 'White',
 'sex': 'Male',
 'capital-gain': 0,
 'capital-loss': 0,
 'hours-per-week': 45,
 'native-country': 'United-States'}

In [38]:
import pandas as pd
import json

datos_json = {'age': 39,
 'workclass': 'Private',
 'fnlwgt': 346478,
 'education': 'Some-college',
 'education-num': 10,
 'marital-status': 'Never-married',
 'occupation': 'Exec-managerial',
 'relationship': 'Not-in-family',
 'race': 'White',
 'sex': 'Male',
 'capital-gain': 0,
 'capital-loss': 0,
 'hours-per-week': 45,
 'native-country': 'United-States'}
DF=pd.DataFrame([datos_json])

prediction=modelo.predict_proba(DF)
prediction[0][1]

0.1249753716032663

In [11]:
datos_testeo.to_json()

'{"age":{"3419":39},"workclass":{"3419":"Private"},"fnlwgt":{"3419":346478},"education":{"3419":"Some-college"},"education-num":{"3419":10},"marital-status":{"3419":"Never-married"},"occupation":{"3419":"Exec-managerial"},"relationship":{"3419":"Not-in-family"},"race":{"3419":"White"},"sex":{"3419":"Male"},"capital-gain":{"3419":0},"capital-loss":{"3419":0},"hours-per-week":{"3419":45},"native-country":{"3419":"United-States"}}'

### Leer APIs desde Python

Configuraremos la forma en la que Python lee una API:

In [58]:
import requests

def realizar_solicitud_post(url, datos, headers=None):
    """
    Realiza una solicitud POST a una URL específica.

    Parámetros:
    - url (str): La URL del endpoint de la API.
    - datos (dict): Un diccionario con los datos que se enviarán en el cuerpo de la solicitud.
    - headers (dict, opcional): Un diccionario con los headers de la solicitud, como Authorization.

    Retorna:
    - Una tupla (bool, dict/str): El primer elemento indica si la solicitud fue exitosa (True) o no (False).
      El segundo elemento es la respuesta de la API convertida en un diccionario si es posible, o la respuesta cruda en caso de error.
    """
    try:
        response = requests.post(url, json=datos, headers=headers)
        response.raise_for_status()  # Esto lanzará una excepción si la solicitud falla
        try:
            return True, response.json()  # Intenta devolver como JSON
        except ValueError:
            return True, response.text  # Devuelve como texto si no se puede convertir a JSON
    except requests.RequestException as e:
        return False, str(e)  # Devuelve False y el error si la solicitud falló

# Ejemplo de uso
url = 'http://127.0.0.1:8000/adults_model/'
datos_json = {'age': 39,
 'workclass': 'Private',
 'fnlwgt': 346478,
 'education': 'Some-college',
 'education-num': 10,
 'marital-status': 'Never-married',
 'occupation': 'Exec-managerial',
 'relationship': 'Not-in-family',
 'race': 'White',
 'sex': 'Male',
 'capital-gain': 10000,
 'capital-loss': 0,
 'hours-per-week': 45,
 'native-country': 'United-States'}
#headers = {'Authorization': 'Bearer tu_token_aqui'}

exito, respuesta = realizar_solicitud_post(url, datos_json)

if exito:
    print('Solicitud exitosa. Respuesta:', respuesta)
else:
    print('Error en la solicitud:', respuesta)


Solicitud exitosa. Respuesta: {'prediction': 0.796116570554393}
